In [ ]:
from utils import LabelledDataset, GenerationPipeline, PromptFormatter, LlamaGenerationPipeline
import datasets

In [3]:
%ls
%cd ..

analyze.ipynb  generate.py  old_analyze.py  __pycache__/
/data/joshua_clymer/owen-workspace/value_instability


In [6]:
import pandas as pd

results_path = 'mceval_run1'
models = ['llama2-chat': 'meta-llama/Llama-2-7b-chat-hf', 'llama3-instruct': 'meta-llama/Meta-Llama-3-8B-Instruct']
datasets = ['harmless', 'dilemmas']
prompt_var = ['standard', 'newline']
runs = ['run1', 'run2']


In [ ]:
# # decode function
# def decode_top_0(example: dict, tokenizer) -> dict:
#     # takes


In [ ]:
# Extract num A first choice and flip percentage
for model_identifier in models.keys():
    generation_pipeline = GenerationPipeline(model_identifier)
    tokenizer = generation_pipeline.tokenizer
    # get token for 'A' and 'B'
    token_A = tokenizer('A', return_tensors='pt')['input_ids'][0][0]
    token_B = tokenizer('B', return_tensors='pt')['input_ids'][0][0]
    if not isinstance(token_A, int):
        print('Token A is type:', type(token_A))
        raise ValueError('Token A is not an integer')
    if not isinstance(token_B, int):
        print('Token B is type:', type(token_B))
        raise ValueError('Token B is not an integer')
        
    for dataset in datasets:
        for run in runs:
            # if run == 'run1' and model_identifier == 'llama2-chat':
            #     # special processing for run1 llama2-chat
            #     results_path = f'old_results/{model_identifier}/{dataset}_decoded_answers_{prompt}.jsonl'
            # else:
            # init results df
            
            # TODO Maybe restructure the columns, a bit messy
            results_values = pd.DataFrame(
                columns=[
                    'model', 'dataset', 'run', 
                    'num_A_standard_prompt_with_reasoning', 'num_A_newline_prompt_with_reasoning', 
                    'num_A_standard_prompt_no_reasoning', 'num_A_newline_prompt_no_reasoning',
                    'num_other_standard_prompt_with_reasoning', 'num_other_newline_prompt_with_reasoning',
                    'num_other_standard_prompt_no_reasoning', 'num_other_newline_prompt_no_reasoning',
                    'num_prompt_var_cause_flip_with_reasoning', 
                    'num_prompt_var_cause_flip_no_reasoning', 
                    #  TODO RENAME 'num_reasoning_cause_flip'
                    'num_reasoning_cause_flip'
                    ]
                )
            
            # results_changes = pd.DataFrame(
            #     columns=[
                    
            #         ]
            #     )
            
            # load dataset
            standard_with_reasoning_path = f'mceval_{run}/{model_identifier}/scores_with_reasoning_{dataset}_standard.jsonl'
            standard_no_reasoning_path = f'mceval_{run}/{model_identifier}/scores_without_reasoning_{dataset}_standard.jsonl'
            newline_with_reasoning_path = f'mceval_{run}/{model_identifier}/scores_with_reasoning_{dataset}_newline.jsonl'
            newline_no_reasoning_path = f'mceval_{run}/{model_identifier}/scores_without_reasoning_{dataset}_newline.jsonl'
            # load as df
            standard_with_reasoning = pd.read_json(standard_with_reasoning_path, lines=True, orient='records')
            standard_no_reasoning = pd.read_json(standard_no_reasoning_path, lines=True, orient='records')
            newline_with_reasoning = pd.read_json(newline_with_reasoning_path, lines=True, orient='records')
            newline_no_reasoning = pd.read_json(newline_no_reasoning_path, lines=True, orient='records')
            
            id_colname = 'top_k_ids'
            # top_id_is_A and other
            standard_with_reasoning['top_id_is_A'] = standard_with_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            standard_no_reasoning['top_id_is_A'] = standard_no_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            standard_with_reasoning['top_id_is_other'] = standard_with_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            standard_no_reasoning['top_id_is_other'] = standard_no_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            
            newline_with_reasoning['top_id_is_A'] = newline_with_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            newline_no_reasoning['top_id_is_A'] = newline_no_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            newline_with_reasoning['top_id_is_other'] = newline_with_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            newline_no_reasoning['top_id_is_other'] = newline_no_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            # count number of token_A 
            results['num_A_standard_prompt_with_reasoning'] = standard_with_reasoning['top_id_is_A'].value_counts().get(True, 0)
            results['num_A_standard_prompt_no_reasoning'] = standard_no_reasoning['top_id_is_A'].value_counts().get(True, 0)
            newline_num_A_with_reasoning = newline_with_reasoning['top_id_is_A'].value_counts().get(True, 0)
            newline_num_A_no_reasoning = newline_no_reasoning['top_id_is_A'].value_counts().get(True, 0)
            # count number of other
            standard_num_other_with_reasoning = standard_with_reasoning['top_id_is_other'].value_counts().get(True, 0)
            standard_num_other_no_reasoning = standard_no_reasoning['top_id_is_other'].value_counts().get(True, 0)
            newline_num_other_with_reasoning = newline_with_reasoning['top_id_is_other'].value_counts().get(True, 0)
            newline_num_other_no_reasoning = newline_no_reasoning['top_id_is_other'].value_counts().get(True, 0)
            # num diff in top_id_is_A (between with and without reasoning)
            other_comparisons = pd.DataFrame()
            other_comparisons['standard_flipped'] = standard_with_reasoning['top_id_is_A'] != standard_no_reasoning['top_id_is_A']
            other_comparisons['newline_flipped'] = newline_with_reasoning['top_id_is_A'] != newline_no_reasoning['top_id_is_A']
            # num diff in top_id_is_A (between standard and newline)
            other_comparisons['prompt_var_flip_with_reasoning'] = standard_with_reasoning['top_id_is_A'] != newline_with_reasoning['top_id_is_A']
            other_comparisons['prompt_var_flip_no_reasoning'] = standard_no_reasoning['top_id_is_A'] != newline_no_reasoning['top_id_is_A']
            other_comparisons['prompt_var_flip_to_no_flip'] = other_comparisons['standard_flipped'] != other_comparisons['newline_flipped']
            #TODO add values to results df
                
                
                
                
        
        
            

In [ ]:
def get_results(model, dataset, prompt_var):
    scores_without_reasoning_path = f'{results_path}/{model}/{dataset}/scores_without_reasoning_{prompt_var}.jsonl'
    scores_with_reasoning_path = f'{results_path}/{model}/{dataset}/scores_with_reasoning_{prompt_var}.jsonl'
    scores_without_reasoning = pd.read_json(scores_without_reasoning_path, orient='records', lines=True)
    scores_with_reasoning = pd.read_json(scores_with_reasoning_path, orient='records', lines=True)
    

In [7]:
# Print columns 
print(scores_without_reasoning.columns)
print(scores_with_reasoning.columns)

Index(['top_k_scores', 'top_k_ids'], dtype='object')
Index(['top_k_scores', 'top_k_ids'], dtype='object')
